In [307]:
import os
from os.path import join as pathjoin

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

In [308]:
# All the code for rasterizing image:

def crop_image(img, y, x, h, w):
    """
    Crop the image with given top-left anchor and corresponding width & height
    :param img: image to be cropped
    :param y: height of anchor
    :param x: width of anchor
    :param h: height of the patch
    :param w: width of the patch
    :return:
    """
    if len(img.shape) == 2:
        return img[y:y+w, x:x+h]
    else:
        return img[y:y+w, x:x+h, :]


def make_grid(tile_size, patch_size, overlap=0):
    """
    Extract patches at fixed locations. Output coordinates for Y,X as a list (not two lists)
    :param tile_size: size of the tile (input image)
    :param patch_size: size of the output patch
    :param overlap: #overlapping pixels
    :return:
    """
    max_h = int(tile_size[0] - patch_size[0])
    max_w = int(tile_size[1] - patch_size[1])

    if max_h > 0 and max_w > 0:
        h_step = int(np.ceil(tile_size[0] / (patch_size[0] - overlap)))
        w_step = int(np.ceil(tile_size[1] / (patch_size[1] - overlap)))
    else:
        h_step = 1
        w_step = 1
    patch_grid_h = np.floor(np.linspace(0, max_h, h_step)).astype(np.int32)
    patch_grid_w = np.floor(np.linspace(0, max_w, w_step)).astype(np.int32)

    y, x = np.meshgrid(patch_grid_h, patch_grid_w)

    return list(zip(y.flatten(), x.flatten()))


def patch_tile(rgb, gt, patch_size, pad=0, overlap=0):
    """
    Extract the given rgb and gt tiles into patches
    :param rgb:
    :param gt:
    :param patch_size: size of the patches, should be a tuple of (h, w)
    :param pad: #pixels to be padded around each tile, should be either 
    one element or four elements
    :param overlap: #overlapping pixels between two patches in both vertical
    and horizontal direction
    :return: rgb and gt patches as well as coordinates
    """
    # rgb = misc_utils.load_file(rgb_file)
    # gt = misc_utils.load_file(gt_file)[:, :, 0]
    np.testing.assert_array_equal(rgb.shape[:2], gt.shape)
    grid_list = make_grid(
        np.array(rgb.shape[:2]) + 2 * pad, patch_size, overlap)
    
    for y, x in grid_list:
        rgb_patch = crop_image(
            rgb, y, x, patch_size[0], patch_size[1])
        gt_patch = crop_image(
            gt, y, x, patch_size[0], patch_size[1])

        yield rgb_patch, gt_patch, y, x


def read_geotiff(geotiff_path):
    """Read geotiff, return reshaped image and metadata."""
    with rasterio.open(geotiff_path, 'r') as src:
        img = src.read()
        img_meta = src.meta
    return reshape_as_image(img), img_meta

def read_labels(labels_path, geotiff_crs, plant_id):
    """Read geojson labels and convert projection, return geopandas dataframe."""
    labels = gpd.read_file(labels_path)
    labels = labels[labels.geometry.notnull()]#[labels.building == 'yes']
    lb = labels.to_crs({'init': geotiff_crs['init']})
    lb_plant = lb.loc[lb['id'] == plant_id,:]
    return lb_plant

def make_dir_if_not_exists(path, return_path=False):
    if not os.path.exists(path):
        os.makedirs(path)
    if return_path:
        return path

def save_image(img, path, name):
    make_dir_if_not_exists(path)
    data = Image.fromarray(img.astype(np.uint8))
    data.save(pathjoin(path, name))

def rasterize_labels(labels, img_size,img_meta):
    """
    Draw rasterized labeled imagery based on corresponding geotiff image size.
    :param labels: geopandas dataframe, must have 'geometry' column with Polygon objects
    :img_size: corresponding geotiff image size
    """
    new_polygons = []

    for _, row in labels.iterrows():
        if isinstance(row['geometry'], Polygon):
            new_polygons.append(convert_polygon(
                row['geometry'], img_meta['transform']))
        elif isinstance(row['geometry'], MultiPolygon):
            for poly in list(row['geometry']):
                new_polygons.append(convert_polygon(
                    poly, img_meta['transform']))
        else:
            continue
    return rasterize(shapes=new_polygons, out_shape=img_size)

def convert_polygon(rowcol_polygon, transform):
    """
    Convert polygons from geojson rowcol coordinates to pixel positions
    :param rowcol_polygon: geojson polygon(s)
    :param transform: affine.Affine object, read from geotiff meta
    """
    polygon_points = []

    for point in np.array(rowcol_polygon.exterior.coords):
        # transform rowcol coords to geotiff crs, using reverse affine transformation
        polygon_points.append(~transform * point)

    return Polygon(polygon_points)

In [309]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from OSMPythonTools.api import Api

'''This file contains the following functions:

1. get_ids(geojson_file_path)

Given a file_path, gets the ids for all the *ways* it contains
Each id will be an input to the next function:

2. get_coord(way_id)

Returns a list with the coordinates of the vertices of this way.
This function can be mapped on the output of the previous function
to return a list of the coordinates of each *way*.

The output of the get_coord function serves as an input to the next function:

3. bound_box(list_of_coordinates)

Constructs a box which bounds a *way* object and returns it as a Polygon object.

'''
def get_ids(geojson_file_path):
    '''Given a geojson file path extracted from OCM
    with different identified structures (*ways*),
    this function returns a list with their ids.'''

    #Read geojson file
    geojson = gpd.read_file(geojson_file_path)

    #Get the id colum and transform it in a list
    ids = list(geojson.loc[1:,'id'])
    names = list(geojson.loc[1:,'name'])
    dic = {}
    for i in range(len(ids)):
        if (ids[i].split('/')[0] == 'way'):
            dic[ids[i]] = names[i]
    #print(ids)
    #print(names)
    #Return list with ids
    return dic

def get_coord(uniq_id):
    '''Given the id of one identified structure (*way*),
    from OCM, this function returns a list with its coordinates'''

    #Define empty list to store coordinates
    coord = []

    #Execute query using OCM API
    #Requires command
    #from OSMPythonTools.api import Api
    api = Api()
    query_results = api.query(uniq_id)
    idType = uniq_id.split('/')[0]
    nodes = []
    if (idType == 'way'):
        nodes = query_results.nodes()
    elif(idType == 'relation'):
        nodes = query_results.members()
    #Get the coordinates for each node and store in list
    for node in nodes:
        coord.append(node.geometry()['coordinates'])

    return(coord)


def bound_box(list_of_coordinates):
    '''Given a list of coordinates
    (such as the output of function get_coords)
    returns a rectangle which contains the polygon
    defined by the coordinates.
    This rectangle is itself returned as a polygon object.
    Note that this is not the minimum bound box,
    but rather the minimum bound box which is parallel to the x and y axis (parallels and meridians).

    Requires
    from shapely.geometry import Polygon.'''

    #Collects x's and y's of polygons vertices
    x = []
    y = []
    for vertex in list_of_coordinates:
        x.append(vertex[0])
        y.append(vertex[1])

    #Define minimum and maximum values of x and y
    min_x = min(x)
    min_y = min(y)
    max_x = max(x)
    max_y = max(y)

    #Define vertices of box
    #Note that these are not point objects
    #as polygon objects in shapely are not created from point objects
    point_A = (min_x, min_y)
    point_B = (min_x, max_y)
    point_C = (max_x, max_y)
    point_D = (max_x, min_y)

    #Define bound box object as Polygon
    box = Polygon([point_A, point_B, point_C, point_D])

    #Return bound_box
    return(box)

In [310]:
def get_rast_image(filename, img_path,json_filepath, output_path, plant_id):
    patch_size = (500,500)
    # This will read the image and the meta data

    img,img_meta = read_geotiff(img_path)
    # get the image sizes
    img_size = (img_meta['height'], img_meta['width'])
    # readthe crs
    #labels = read_labels('jsons/{}.json'.format(filename),  img_meta['crs'])
    labels = read_labels(json_filepath, img_meta['crs'], plant_id)
    gt =rasterize_labels(labels, img_size,img_meta)
    img_patches_dir = output_path+'rgb'
    gt_patches_dir = output_path+'gt'
    loc='nia'
    idx=0
    for img_patch, gt_patch, y, x in patch_tile(img, gt, patch_size):
        idx+=1
        img_patchname='img-{}-{}.png'.format(filename,idx)
        gt_patchname='gt-{}-{}.png'.format(filename,idx)
        save_image(img_patch, img_patches_dir, img_patchname)
        save_image(gt_patch*255, gt_patches_dir, gt_patchname)
        break



In [311]:
import ee
import os
import time
from ee import batch
ee.Initialize()

def getTask(coords, input_id_name):
    # takes as input coordinates of the boundary to export to 
    # google cloud bucket
    geom = ee.Geometry.Rectangle([coords[0],coords[1],coords[2],coords[3]]);
    collection = ee.ImageCollection("USDA/NAIP/DOQQ") \
                .filter(ee.Filter.date('2015-01-01', '2017-12-31'));
    trueColor = collection.select(['R', 'G', 'B','N'])
    trueColorVis = {
      min: 0.0,
      max: 255.0,
    }
    image = collection.sort('system:index', False).mosaic()
    image = image.clip(geom)
    image.projection()

    task = ee.batch.Export.image.toCloudStorage(image=image,
                                        region=image.geometry().bounds().\
                                        getInfo()['coordinates'],
                                        description='power_plant',
                                        outputBucket='earth_engine_data',
                                        fileNamePrefix=input_id_name,
                                        scale=1)
    return task

def downloadGStorage(buck_name, local_addr):
    os.system('export GOOGLE_APPLICATION_CREDENTIALS=\"key.json\"')
    os.system('gsutil cp -r '+ buck_name + ' ' + local_addr)
    pass

WORKING CODE FOR THE TOOL BELOW:

Main demonstration:

https://overpass-turbo.eu/s/QBq
 
The below geojson was taken from the link above:

simply open the link and open and exprot the geojson


For a solar panel ground truth set:
http://overpass-turbo.eu/s/R6y


In [317]:
#here print to see available power plants
dic_idToName = get_ids(allPlantsGeojson)
#print(dic_idToName)

In [327]:
#need to wait for a while to have it for downloading
#We are now using record of power plants in eastern US
allPlants= "eastern_US"
allPlantsGeojson = "jsons/"+allPlants+".geojson"
#identified by id

stored = []
for i in range(60,65):
    #functionality 1, type way id here to grab certain power plant, way id can be looked up from "dic_idToName" dictionary
    plant_id = list(dic_idToName.keys())[i] #'way/40238958'
    print(plant_id)
    plant_name = dic_idToName[plant_id] #Mercer Generating Station
    print(plant_name)
    imageName = plant_id.replace('/', '-') + "-" + str(plant_name)
    stored.append((plant_id, imageName))
    allCoords = get_coord(plant_id)
    bBoxCoords = np.array(bound_box(allCoords).exterior)
    res = ggeToGoogleDr(allPlants, allPlantsGeojson, bBoxCoords, imageName)

way/69342304
Intercession City Plant DUKE
way/69515745
Indian River Power Station
way/69962263
Cane Island Power Park
way/69966315
DeCew I Generating Station
way/70080734
Indian River Generating Station


In [328]:
# step 2
googleDrToLocal(allPlants, 'gs://earth_engine_data', 'gee_images/')

In [329]:
# step 3
# this will fetch the image
# and rasterize the image
for plant_id, imageName in stored:
    get_rast_image(imageName,'gee_images/earth_engine_data/'+imageName+'.tif', allPlantsGeojson, 'images/patches/', plant_id)

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<

Image processed can be found in images/patches.

Functionality 1 DONE

In [247]:
# functionality 2: pick random image with certain coords

In [248]:
#need to wait for downloading from GGE
randImageName = "random_one"
randCoords = np.array([
    [-78.085,  35.391],
    [-78.085,  35.395],
    [-78.080,  35.395],
    [-78.080,  35.391],
    [-78.085,  35.391]])
res = ggeToGoogleDr(allPlants, allPlantsGeojson, randCoords, randImageName )

In [249]:
# step 2
googleDrToLocal(allPlants, 'gs://earth_engine_data', 'gee_images/')

In [250]:
# step 3
# this will fetch the image
# and rasterize the image to see if certain plant in it
# this one has no power plant in it, so it will be rasterized as a black image
get_rast_image(randImageName,'gee_images/earth_engine_data/'+randImageName +'.tif', allPlantsGeojson, 'images/patches/', plant_id)

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
